In [70]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import optuna
import numpy as np
import time

In [71]:
df = pd.read_csv('Clean_Data.csv', sep=';')
df_no_cat = df.drop(columns=['student_id','student_email','group_id','group_title','prep_email','date_first_l','feat_group_format','feat_payment_type','feat_payment_place'])
df_cat = df[['feat_group_format','feat_payment_type','feat_payment_place']]

In [72]:
encoded_ohe_df = df_no_cat
ohe = OneHotEncoder(sparse_output=False, dtype=int)

ohe_df_1 = pd.DataFrame(ohe.fit_transform(df_cat[['feat_payment_type']]), columns=ohe.get_feature_names_out(['feat_payment_type']))
encoded_ohe_df = pd.concat([encoded_ohe_df, ohe_df_1], axis=1) # В том случае, если хотим добавить закодированные данные
ohe = OneHotEncoder(sparse_output=False, dtype=int)
ohe_df_2 = pd.DataFrame(ohe.fit_transform(df_cat[['feat_payment_place']]), columns=ohe.get_feature_names_out(['feat_payment_place']))
encoded_ohe_df = pd.concat([encoded_ohe_df, ohe_df_2], axis=1) # В том случае, если хотим добавить закодированные данные
ohe = OneHotEncoder(sparse_output=False, dtype=int)
ohe_df_3 = pd.DataFrame(ohe.fit_transform(df_cat[['feat_group_format']]), columns=ohe.get_feature_names_out(['feat_group_format']))
encoded_ohe_df = pd.concat([encoded_ohe_df, ohe_df_3], axis=1) # В том случае, если хотим добавить закодированные данные

In [73]:
encoded_ohe_df

,feat_class,feat_group_weeks,feat_group_size,feat_attendance,feat_HW,feat_HW_right,feat_theory_read,feat_TK,feat_LB,feat_tickets_we_need,...,feat_payment_place_Cайт,feat_payment_place_ДОД,feat_payment_place_ИK,feat_payment_place_ИК,feat_payment_place_Навигатор,feat_group_format_Блендед,feat_group_format_Микро,feat_group_format_Мини,feat_group_format_Онлайн+Класс,feat_group_format_Стандарт
0,10,17,7,1.00,0.790759,0.732470,0.32,1.0,3,0,...,0,0,1,0,0,1,0,0,0,0
1,10,17,7,0.43,0.472222,0.404222,0.31,0.0,1,0,...,0,0,1,0,0,1,0,0,0,0
2,10,17,7,1.00,0.253038,0.239595,0.22,1.0,3,3,...,0,0,1,0,0,1,0,0,0,0
3,10,17,7,1.00,0.813291,0.651681,0.41,1.0,3,1,...,0,0,1,0,0,1,0,0,0,0
4,10,17,7,1.00,0.241139,0.205468,0.56,1.0,3,3,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67333,9,21,11,0.90,0.261207,0.215809,0.06,0.4,1,0,...,0,0,1,0,0,0,0,1,0,0
67334,9,21,11,0.95,0.417586,0.357788,0.23,0.5,2,0,...,0,0,1,0,0,0,0,1,0,0
67335,9,21,11,1.00,0.651552,0.532472,1.00,0.8,2,0,...,0,0,1,0,0,0,0,1,0,0
67336,9,21,11,1.00,0.815000,0.560593,1.00,1.0,0,0,...,0,0,1,0,0,0,0,1,0,0


In [74]:
corr_matrix = encoded_ohe_df.corr()
# Выделяем строку корреляции с целевой переменной
target_corr = corr_matrix['target'].abs().sort_values(ascending=False)

# Выбираем признаки с корреляцией выше порога (например, 0.3)
strong_features = target_corr[target_corr > 0.05].index.tolist()

print(f"Лучшие признаки: {strong_features}")

Лучшие признаки: ['target', 'feat_good_agr_before', 'feat_attendance', 'feat_LB', 'feat_HW', 'feat_HW_right', 'feat_theory_read', 'feat_TK', 'feat_payment_type_Позтапная', 'feat_payment_type_Единовременная', 'feat_tickets_mb_term', 'feat_term_perc', 'feat_payment_place_Навигатор', 'feat_prep_HW']


In [75]:
df_few = encoded_ohe_df[strong_features]

In [76]:
features = df_few.drop('target', axis=1)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)
scaled_features_df = pd.DataFrame(scaled_features, columns=features.columns)
scaled_data = pd.concat([scaled_features_df, df_few['target'].reset_index(drop=True)], axis=1)
scaled_data.head()

X = scaled_data.drop('target', axis=1)
y = scaled_data['target']

## 1. KNN with unbalanced data

In [77]:
kfold = KFold(n_splits=10, shuffle=True)
skf = StratifiedKFold(n_splits=5, shuffle=True)

In [78]:
X_unbalanced_train, X_unbalanced_test, y_unbalanced_train, y_unbalanced_test = train_test_split(X,y,random_state=42, stratify=y)

In [79]:
start = time.time()
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_unbalanced_train, y_unbalanced_train)
print(f'Время обучения модели: {round(time.time() - start,2)} cекунд')

Время обучения модели: 0.08 cекунд


In [80]:
y_unbalanced_pred = knn.predict(X_unbalanced_test)
print(classification_report(y_unbalanced_test, y_unbalanced_pred))

              precision    recall  f1-score   support

           0       0.93      0.98      0.95     14574
           1       0.79      0.52      0.62      2261

    accuracy                           0.92     16835
   macro avg       0.86      0.75      0.79     16835
weighted avg       0.91      0.92      0.91     16835



In [81]:
start = time.time()
print(f'f-1 score: {round(cross_val_score(knn, X, y, cv=skf, scoring='f1').mean(),3)} \nВремя кроссвалидации: {round(time.time() - start,2)} секунд')

f-1 score: 0.633 
Время кроссвалидации: 13.71 секунд


## 2. KNN with oversampling

In [82]:
from imblearn.over_sampling import RandomOverSampler

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42, stratify=y)

In [84]:
ros = RandomOverSampler()
X_over_train, y_over_train = ros.fit_resample(X_train, y_train)

In [85]:
start = time.time()
knn_over = KNeighborsClassifier(n_neighbors=5)
knn_over.fit(X_over_train, y_over_train)
print(f'Время обучения модели: {round(time.time() - start,2)} cекунд')

Время обучения модели: 0.17 cекунд


In [86]:
y_over_pred = knn_over.predict(X_test)
print(classification_report(y_test, y_over_pred))

              precision    recall  f1-score   support

           0       0.95      0.86      0.90     14574
           1       0.44      0.71      0.54      2261

    accuracy                           0.84     16835
   macro avg       0.69      0.78      0.72     16835
weighted avg       0.88      0.84      0.85     16835



## 3. KNN with undersampling

In [87]:
from imblearn.under_sampling import RandomUnderSampler

In [88]:
rus = RandomUnderSampler()
X_under, y_under = rus.fit_resample(X, y)

X_under_train, X_under_test, y_under_train, y_under_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42)

In [89]:
start = time.time()
knn_under = KNeighborsClassifier(n_neighbors=14, weights = 'distance')
knn_under.fit(X_under_train, y_under_train)
print(f'Время обучения модели: {round(time.time() - start,2)} cекунд')

Время обучения модели: 0.02 cекунд


In [90]:
y_under_pred = knn_under.predict(X_under_test)
print(classification_report(y_under_test, y_under_pred))

              precision    recall  f1-score   support

           0       0.77      0.86      0.82      1801
           1       0.85      0.75      0.80      1817

    accuracy                           0.81      3618
   macro avg       0.81      0.81      0.81      3618
weighted avg       0.81      0.81      0.81      3618



In [91]:
start = time.time()
print(f'f-1 score: {round(cross_val_score(knn_under, X_under, y_under, cv = skf, scoring='f1').mean(),3)} \nВремя кроссвалидации: {round(time.time() - start,2)} секунд')

f-1 score: 0.804 
Время кроссвалидации: 1.73 секунд


## Model optimization

In [46]:
rus = RandomUnderSampler()
X_under, y_under = rus.fit_resample(X, y)
X_under_train, X_under_test, y_under_train, y_under_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42)

# Функция оптимизации гиперпараметров
def objective(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 20)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    
    # Создание и обучение модели
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights)
    knn.fit(X_under_train, y_under_train)
    
    # Предсказание на тестовых данных
    y_pred = knn.predict(X_under_test)
    
    # Вычисление точности
    f1 = f1_score(y_under_test, y_pred)
    return f1

# Создание и запуск исследования Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Вывод лучших гиперпараметров и точности
print('Best hyperparameters:', study.best_params)
print('Best accuracy:', study.best_value)

[I 2025-04-11 14:26:17,021] A new study created in memory with name: no-name-f0c9fb98-6cf0-41f7-828a-a627a67fb61d
[I 2025-04-11 14:26:17,375] Trial 0 finished with value: 0.7972933215651662 and parameters: {'n_neighbors': 14, 'weights': 'distance'}. Best is trial 0 with value: 0.7972933215651662.
[I 2025-04-11 14:26:17,805] Trial 1 finished with value: 0.7850916741809438 and parameters: {'n_neighbors': 20, 'weights': 'uniform'}. Best is trial 0 with value: 0.7972933215651662.
[I 2025-04-11 14:26:18,146] Trial 2 finished with value: 0.7863100373885533 and parameters: {'n_neighbors': 5, 'weights': 'uniform'}. Best is trial 0 with value: 0.7972933215651662.
[I 2025-04-11 14:26:18,465] Trial 3 finished with value: 0.7929540860525556 and parameters: {'n_neighbors': 8, 'weights': 'distance'}. Best is trial 0 with value: 0.7972933215651662.
[I 2025-04-11 14:26:18,784] Trial 4 finished with value: 0.7730061349693251 and parameters: {'n_neighbors': 4, 'weights': 'uniform'}. Best is trial 0 with

Best hyperparameters: {'n_neighbors': 14, 'weights': 'distance'}
Best accuracy: 0.7972933215651662
